# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

In [5]:
! git clone https://github.com/DS3001/wrangling

fatal: destination path 'wrangling' already exists and is not an empty directory.


**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

1. The paper is about data tidying which is a subset of data cleaning that makes it easy to "manipulate, model and visualize" data sets.
2. The tidy data standard provides a way to organize data values within a dataset.

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [ ]:
# Question 1
# loading and viewing data
data1 = pd.read_csv('https://raw.githubusercontent.com/vrhughes/wrangling/main/assignment/data/airbnb_hw.csv', low_memory = False)
print(data1.shape, '\n') # dimensions
print(data1.dtypes, '\n') # types
print(data1.columns, '\n') # col names
data1.head()

In [ ]:
# Question 1
# assign variable name and look for unique values to see problems easier
price = data1['Price']
print(price.unique(), '\n')

# reassign getting rid of commas
price = price.str.replace(',', '')
print(price.unique(), '\n')

# find out type of price, if it is not price then will need to coerce
print('Before coercion: \n', price.describe(), '\n')

# coerce price to numeric from object type
price = pd.to_numeric(price, errors = 'coerce')

# see after coercion and check again for unique values
print('After coercion: \n', price.describe(), '\n', price.unique(), '\n')

# missing dummy to see if any values are missing
data1['price_missing'] = price.isnull()
print('Missing: \n', sum(data1['price_missing']), '\n')

# Since no values were missing I decided this data was cleaned to the best of my ability

In [ ]:
# Question 2
# loading in and viewing data
sharks = pd.read_csv('https://raw.githubusercontent.com/vrhughes/wrangling/main/assignment/data/sharks.csv', low_memory = False)
print(sharks.shape, '\n') # dimensions
print(sharks.dtypes, '\n') # types
sharks.head()

# getting a better idea of 'Type' values
print(sharks['Type'].unique(), '\n')
print(sharks['Type'].value_counts(), '\n')

# consolidate 'Type' categories
sharks['Type'] = sharks['Type'].replace(['Sea Disaster', 'Boat', 'Boating', 'Boatomg'], 'Watercraft')
sharks['Type'] = sharks['Type'].replace(['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan)

# visualize consolidation
print(sharks['Type'].value_counts(), '\n')


In [ ]:
# Question 3
# load data and take a peek
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
d3 = pd.read_csv(url, low_memory = False) # Pandas downloads and loads the .csv file for you
d3.head()

In [ ]:
# Question 3
# making a dummy variable
released = d3['WhetherDefendantWasReleasedPretrial']

# viewing released data
print(released.unique(), '\n')
print(released.value_counts(), '\n')
print(released.describe(), '\n')

# Replacing 9s with nan
released = released.replace(9, np.nan)
print(released.unique(), '\n')
print(released.value_counts(), '\n')
print(released.describe(), '\n')

# Reinserting the cleaned data
d3['WhetherDefendantWasReleasedPretrial'] = released
del released # getting rid of dummy var
d3['WhetherDefendantWasReleasedPretrial'].value_counts() # Checking that everything worked

In [28]:
# Question 4
# making dummy variables and viewing them
time = d3['ImposedSentenceAllChargeInContactEvent']
category = d3['SentenceTypeAllChargesAtConvictionInContactEvent']
print(time.unique(), '\n')
print(category.unique(), '\n')
print(time.value_counts(), '\n')
print(category.value_counts(), '\n')

# from observing the data and codebook
# category: 0 = probation/no incarceration, 1 = jail up to 12 months, 2 = prison (1+ yrs), 4 = dismissed, deferred, 9 = not applicable/unknown
# time: some data are ' '
time = pd.to_numeric(time, errors = 'coerce')

# see how many vals are missing
missing = time.isnull()
print(sum(missing), '\n')

time = time.mask(category == 9, np.nan)
time = time.mask(category == 4, 0)

# see how many vals are missing still
missing = time.isnull()
print(sum(missing), '\n')

# reinstate time into dataframe
d3['ImposedSentenceAllChargeInContactEvent'] = time
del time, category


[' ' '60' '12' '.985626283367556' '36' '6' '24' '5.91375770020534' '120'
 '72' '11.9917864476386' '0' '2.95687885010267' '84' '108' '300' '240'
 '180' '4' '96' '2' '54' '.328542094455852' '44' '5' '115' '132' '48'
 '258' '34' '76' '.164271047227926' '.131416837782341' '111' '9' '3'
 '1.97125256673511' '36.9856262833676' '.0657084188911704'
 '35.4928131416838' '106.492813141684' '8' '35' '18.3141683778234' '480'
 '32' '93' '234' '732' '1.16427104722793' '4.6570841889117' '21' '7'
 '4.49281314168378' '18' '600' '43.1642710472279' '179' '52' '30' '20'
 '192' '702' '14' '55' '53' '11.9055441478439' '114' '35.0061601642711'
 '68' '.657084188911704' '46.6242299794661' '102' '65' '200' '57'
 '24.3285420944559' '12.1642710472279' '117' '81.4928131416838'
 '22.4928131416838' '1980' '3.6570841889117' '56' '10' '2.79260780287474'
 '1' '47' '22' '1500' '40' '284' '11' '118' '42' '162' '156'
 '47.2956878850103' '105' '51' '246' '29' '75' '324' '360'
 '34.4804928131417' '120.328542094456' '59.926078

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?